# TD Ameritrade Dashboard

#### Import Statements & Importing Variables from config.ipynb

In [ ]:
# from config import consumer_key, redirect_uri, json_path
# Jupyter Notebook Notation, pulling from 'config.ipynb'
%store -r consumer_key
%store -r redirect_uri
%store -r json_path
%store -r google_cloud_api_key
%store -r scope
# non pickleable variables here were imported at the bottom of this cell
# %store -r credentials
# %store -r gc
%store -r spreadsheet_key


import requests
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
from td.client import TDClient
import platform
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import time


# non pickleable variables
credentials = ServiceAccountCredentials.from_json_keyfile_name('jsonFileFromGoogle.json', scopes=scope)
gc = gspread.authorize(credentials)

In [ ]:
# #storing todays date
today = time.time()
today = int(today)
today = today*1000

tomorrow = (today+86400)*1000

%store today
%store tomorrow

#### Assigned variable json_path from config.ipynb. The variable decides a file path based on operating system. I have two computers, one Mac and one PC. So the file paths are dependent on which computer I am on.

In [ ]:
json_path

#### Collecting API Token

In [ ]:
# Create a new instance of the client
td_client = TDClient(client_id = consumer_key, redirect_uri = redirect_uri, credentials_path = json_path)

In [ ]:
# Log in to a new session -- every 90 days -- comment out to avoid it autorunning with kernel restart
td_client.login()

#### Build dictionaries with values based on current account values

In [ ]:
# Orders and Positions for account(s)
orders_and_positions = td_client.get_accounts(account = 'all', fields = ['orders',
                                                                         'positions'])
orders_and_positions

#### Assigning account positions to proper lists in order to build out account DataFrames 

In [ ]:
cash = []
stock_list = []
option_list = []
for position in orders_and_positions[0]['securitiesAccount']['positions']:
    asset_type = position['instrument']['assetType']
    if asset_type == 'OPTION':
        symbol = position['instrument']['underlyingSymbol']
        quantity = position['longQuantity']
        search_instruments = td_client.search_instruments(symbol = symbol, projection = 'fundamental')
        quotes_response = td_client.get_quotes(instruments = [symbol])
        mark = quotes_response[symbol]['mark']
        beta = search_instruments[symbol]['fundamental']['beta']
        pct_change = position['currentDayProfitLossPercentage']
        print('+++++++++++++')
        print(symbol + ' ' + str(quantity))
        option_list.append({'Symbol':symbol, 'Quantity':quantity, 'Mark':mark, 'Beta':beta, 'Percent Change': pct_change})  
    elif asset_type == 'EQUITY':
        symbol = position['instrument']['symbol']
        quantity = position['longQuantity']
        search_instruments = td_client.search_instruments(symbol = symbol, projection = 'fundamental')
        quotes_response = td_client.get_quotes(instruments = [symbol])
        mark = quotes_response[symbol]['mark']
        beta = search_instruments[symbol]['fundamental']['beta']
        pct_change = position['currentDayProfitLossPercentage']
        print('+++++++++++++')
        print('Market Value')
        print(symbol + ' ' + '$' + str(quantity*mark) + ', ' + 'Beta of ' + str(beta))
        stock_list.append({'Symbol':symbol, 'Quantity':quantity, 'Mark':mark, 'Beta':beta, 'Percent Change': pct_change})       
    else:
        symbol = position['instrument']['symbol']
        quantity = position['longQuantity']
        mark = np.nan
        beta = np.nan
        print('+++++++++++++')
        print(symbol + ' ' + '$' + str(quantity))
        cash.append({'Symbol':symbol, 'Quantity':quantity, 'Mark':mark, 'Beta':beta})

#### Building out DataFrames with lists built above

In [ ]:
stock_list

In [ ]:
stocks = pd.DataFrame(stock_list)
beta_drop = pd.DataFrame(stock_list)
# Adding Value as a column to calculate the position value
stocks['Value'] = stocks['Quantity']*stocks['Mark']

cash = pd.DataFrame(cash)

portfolio = [stocks, cash]
portfolio = pd.concat(portfolio).reset_index().drop(['index'], axis = 1)
portfolio

## Storing List of Portfolio Symbols For News API And Various References

In [ ]:
ticker_list = portfolio['Symbol'].to_list()

%store ticker_list

### Cleaning values
#### Finding Beta values == 0, and replacing them with the mean beta calculated from all other Beta values excluding those positions with values equal to a Beta of 0

In [ ]:
total_value = orders_and_positions[0]['securitiesAccount']['currentBalances']['liquidationValue']
total_value

In [ ]:
drop = beta_drop[beta_drop['Beta']==0].index
beta_drop.drop(drop, inplace = True)
average_beta = beta_drop.Beta.mean()
stocks.loc[stocks.Beta == 0, 'Beta'] = average_beta
stocks['Portfolio Makeup %'] = stocks['Value']/(sum(stocks['Value']) + cash.iloc[0,1])*100
# Looking at volatility levels based on Beta Value
stocks['Risk Level'] = ['High Volatility' if beta >= 2 
                        else 'Medium Volatility' if beta >= 1 and beta < 2 
                        else 'Low Volatility'
                        for beta in stocks.Beta]
stocks['Position Weight'] = ['Over Weight' if weight >= 10
                            else 'Heavy' if weight < 10 and weight >= 7
                            else 'On Par' if weight < 7 and weight >= 4
                            else 'Light' if weight < 4 and weight >= 1
                            else 'Under Weight'
                            for weight in stocks['Portfolio Makeup %']]
stocks

In [ ]:
top_5_by_weight = stocks.nlargest(5, 'Portfolio Makeup %')
top_5_by_weight
%store top_5_by_weight

In [ ]:
%run news_api.ipynb

#### Taking a look at account values DataFrame

In [ ]:
stocks_value = round(sum(stocks['Value']),2)
cash_value = round(cash.iloc[0,1],2)
account_value = total_value

account_values = [{'Stocks':stocks_value, 'Cash':cash_value, 'Account Total': account_value, 'Account Beta': average_beta}]
account_values = pd.DataFrame(account_values)

print(stocks_value, cash_value, account_value)

### Checking For Risk Variations 
---

In [ ]:
%run charting_data.ipynb

### Sending DataFrames to Google Sheets
---

In [ ]:
%store -r news_articles_df

# News Articles
wks_name= 'News Articles'
d2g.upload(news_articles_df, spreadsheet_key, wks_name, credentials=credentials, row_names=False)
print('Data upload success')

In [ ]:
# Portfolio Positions
wks_name= 'Portfolio Positions'
d2g.upload(portfolio, spreadsheet_key, wks_name, credentials=credentials, row_names=False)
print('Data upload success')

In [ ]:
# Account Values
wks_name= 'Account Values'
d2g.upload(account_values, spreadsheet_key, wks_name, credentials=credentials, row_names=False)
print('Data upload success')

In [ ]:
# Stocks Values
wks_name= 'Stocks Values'
d2g.upload(stocks, spreadsheet_key, wks_name, credentials=credentials, row_names=False)
print('Data upload success')

In [ ]:
%store -r charting_data

# Charting Data
wks_name= 'Charting Data'
d2g.upload(charting_data, spreadsheet_key, wks_name, credentials=credentials, row_names=False)
print('Data upload success')